parameters


In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
#data_csv = pd.read_csv('C:\\Users\\86134\\Desktop\\temp.csv', header=None,usecols=[1])
from torch.utils.data import Dataset
import os
from torch.autograd import Variable

In [2]:
class lstm_reg(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=2):
        super(lstm_reg, self).__init__()

        self.rnn = nn.LSTM(input_size, hidden_size, num_layers)  # rnn
        self.reg = nn.Linear(hidden_size, output_size)  # 回归

    def forward(self, x):
        x, _ = self.rnn(x)  # (seq, batch, hidden)
        s, b, h = x.shape
        x = x.view(s * b, h)  # 转换成线性层的输入格式
        x = self.reg(x)
        x = x.view(s, b, -1)
        return x

In [3]:
def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

def normalization(data):
    _range = np.max(data) - np.min(data)
    return (data - np.min(data)) / _range

In [4]:
data_csv=pd.read_csv("industry_ret_data_1500.csv").set_index("Date")
dataset = data_csv.values
dataset = dataset.astype('float32')
#dataset=normalization(dataset)
dataset=standardization(dataset)
print(dataset.shape)
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)


(1522, 433)


In [5]:
output_size=1
epochs=10

In [6]:
from itertools import product

In [7]:
parameters=dict(
    input_size=[3,4,6],
    hidden_size=[3,4,6],
    num_layers=[2,3,4],
    l_rate=[0.1,0.01,0.005,0.001]
)
param_values=[v for v in parameters.values()]
param=list(product(*param_values))
print(param)

[(3, 3, 2, 0.1), (3, 3, 2, 0.01), (3, 3, 2, 0.005), (3, 3, 2, 0.001), (3, 3, 3, 0.1), (3, 3, 3, 0.01), (3, 3, 3, 0.005), (3, 3, 3, 0.001), (3, 3, 4, 0.1), (3, 3, 4, 0.01), (3, 3, 4, 0.005), (3, 3, 4, 0.001), (3, 4, 2, 0.1), (3, 4, 2, 0.01), (3, 4, 2, 0.005), (3, 4, 2, 0.001), (3, 4, 3, 0.1), (3, 4, 3, 0.01), (3, 4, 3, 0.005), (3, 4, 3, 0.001), (3, 4, 4, 0.1), (3, 4, 4, 0.01), (3, 4, 4, 0.005), (3, 4, 4, 0.001), (3, 6, 2, 0.1), (3, 6, 2, 0.01), (3, 6, 2, 0.005), (3, 6, 2, 0.001), (3, 6, 3, 0.1), (3, 6, 3, 0.01), (3, 6, 3, 0.005), (3, 6, 3, 0.001), (3, 6, 4, 0.1), (3, 6, 4, 0.01), (3, 6, 4, 0.005), (3, 6, 4, 0.001), (4, 3, 2, 0.1), (4, 3, 2, 0.01), (4, 3, 2, 0.005), (4, 3, 2, 0.001), (4, 3, 3, 0.1), (4, 3, 3, 0.01), (4, 3, 3, 0.005), (4, 3, 3, 0.001), (4, 3, 4, 0.1), (4, 3, 4, 0.01), (4, 3, 4, 0.005), (4, 3, 4, 0.001), (4, 4, 2, 0.1), (4, 4, 2, 0.01), (4, 4, 2, 0.005), (4, 4, 2, 0.001), (4, 4, 3, 0.1), (4, 4, 3, 0.01), (4, 4, 3, 0.005), (4, 4, 3, 0.001), (4, 4, 4, 0.1), (4, 4, 4, 0.01), 

In [8]:
def train_model(input_size,hidden_size,num_layers,l_rate,output_size=1,epochs=10):
    net = lstm_reg(input_size,hidden_size,output_size,num_layers)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=l_rate)
    data_X, data_Y = create_dataset(dataset,input_size)

    train_X = data_X[:1000]
    train_Y = data_Y[:1000]
    val_X=data_X[1000:1200]
    val_Y=data_Y[1000:1200]
    test_X = data_X[1200:]
    test_Y = data_Y[1200:]

    train_X = train_X.reshape(-1, 433, input_size)
    train_Y = train_Y.reshape(-1, 433, 1)
    test_X = test_X.reshape(-1, 433, input_size)
    test_Y=test_Y.reshape(-1,433,1)
    val_X = val_X.reshape(-1, 433, input_size)
    val_Y = val_Y.reshape(-1, 433, 1)

    train_x = torch.from_numpy(train_X)
    train_y = torch.from_numpy(train_Y)
    test_x = torch.from_numpy(test_X)
    test_y=torch.from_numpy(test_Y)
    val_x=torch.from_numpy(val_X)
    val_y = torch.from_numpy(val_Y)
    #print((train_x.shape))
    tr_loss=[]
    val_loss=[]
    for e in range(epochs):
        var_x = Variable(train_x)
        var_y = Variable(train_y)
        valx=Variable(val_x)
        valy=Variable(val_y)

        # 前向传播
        out = net(var_x)
        val_out=net(valx)
        loss = criterion(out, var_y)
        v_loss=criterion(val_out, valy)
        tr_loss.append(loss)
        val_loss.append(v_loss)
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        #cor.backward()
        optimizer.step()
        #print('Epoch: {}, Loss: {:.5f},val_Loss: {:.5f},cor:{},v_cor:{}'.format(e + 1, loss.data,v_loss,cor,V_cor))

    net = net.eval() # 转换成测试模式

    test_data = Variable(test_x)
    test_tar = Variable(test_y)
    pred_test = net(test_data) # 测试集的预测结果
    test_loss=criterion(pred_test,test_tar)
    print('ave_val_Loss: {:.5f},test_loss:{:.5f}'.format(sum(val_loss)/len(val_loss),test_loss) )
    return sum(val_loss)/len(val_loss),test_loss


In [9]:
total_para=[]

In [10]:
for input_size,hidden_size,num_layers,l_rate in param[0:32]:
    if input_size<=hidden_size:
        print("input_size,hidden_size,num_layers,l_rate:",input_size,hidden_size,num_layers,l_rate)
        varl,testl=train_model(input_size,hidden_size,num_layers,l_rate)
        total_para.append((varl,testl,input_size,hidden_size,num_layers,l_rate))
    else:
        continue



input_size,hidden_size,num_layers,l_rate: 3 3 2 0.1
ave_val_Loss: 1.23825,test_loss:1.16434
input_size,hidden_size,num_layers,l_rate: 3 3 2 0.01
ave_val_Loss: 1.36296,test_loss:1.19176
input_size,hidden_size,num_layers,l_rate: 3 3 2 0.005
ave_val_Loss: 1.23331,test_loss:1.16454
input_size,hidden_size,num_layers,l_rate: 3 3 2 0.001
ave_val_Loss: 1.39799,test_loss:1.34428
input_size,hidden_size,num_layers,l_rate: 3 3 3 0.1
ave_val_Loss: 1.26440,test_loss:1.16364
input_size,hidden_size,num_layers,l_rate: 3 3 3 0.01
ave_val_Loss: 1.27111,test_loss:1.16686
input_size,hidden_size,num_layers,l_rate: 3 3 3 0.005
ave_val_Loss: 1.47617,test_loss:1.32589
input_size,hidden_size,num_layers,l_rate: 3 3 3 0.001
ave_val_Loss: 1.59762,test_loss:1.54151
input_size,hidden_size,num_layers,l_rate: 3 3 4 0.1
ave_val_Loss: 1.28589,test_loss:1.17763
input_size,hidden_size,num_layers,l_rate: 3 3 4 0.01
ave_val_Loss: 1.24578,test_loss:1.16541
input_size,hidden_size,num_layers,l_rate: 3 3 4 0.005
ave_val_Loss: 1

In [11]:
for input_size,hidden_size,num_layers,l_rate in param[32:64]:
    if input_size<=hidden_size:
        print("input_size,hidden_size,num_layers,l_rate:",input_size,hidden_size,num_layers,l_rate)
        varl,testl=train_model(input_size,hidden_size,num_layers,l_rate)
        total_para.append((varl,testl,input_size,hidden_size,num_layers,l_rate))
    else:
        continue

input_size,hidden_size,num_layers,l_rate: 3 6 4 0.1
ave_val_Loss: 1.26090,test_loss:1.16354
input_size,hidden_size,num_layers,l_rate: 3 6 4 0.01
ave_val_Loss: 1.23342,test_loss:1.16325
input_size,hidden_size,num_layers,l_rate: 3 6 4 0.005
ave_val_Loss: 1.23348,test_loss:1.16644
input_size,hidden_size,num_layers,l_rate: 3 6 4 0.001
ave_val_Loss: 1.23217,test_loss:1.16326
input_size,hidden_size,num_layers,l_rate: 4 4 2 0.1
ave_val_Loss: 1.23178,test_loss:1.16563
input_size,hidden_size,num_layers,l_rate: 4 4 2 0.01
ave_val_Loss: 1.23110,test_loss:1.16925
input_size,hidden_size,num_layers,l_rate: 4 4 2 0.005
ave_val_Loss: 1.52881,test_loss:1.39247
input_size,hidden_size,num_layers,l_rate: 4 4 2 0.001
ave_val_Loss: 1.35348,test_loss:1.30358
input_size,hidden_size,num_layers,l_rate: 4 4 3 0.1
ave_val_Loss: 1.23769,test_loss:1.16646
input_size,hidden_size,num_layers,l_rate: 4 4 3 0.01
ave_val_Loss: 1.31355,test_loss:1.17009
input_size,hidden_size,num_layers,l_rate: 4 4 3 0.005
ave_val_Loss: 1

In [12]:
for input_size,hidden_size,num_layers,l_rate in param[64:96]:
    if input_size<=hidden_size:
        print("input_size,hidden_size,num_layers,l_rate:",input_size,hidden_size,num_layers,l_rate)
        varl,testl=train_model(input_size,hidden_size,num_layers,l_rate)
        total_para.append((varl,testl,input_size,hidden_size,num_layers,l_rate))
    else:
        continue

input_size,hidden_size,num_layers,l_rate: 4 6 3 0.1
ave_val_Loss: 1.24604,test_loss:1.16594
input_size,hidden_size,num_layers,l_rate: 4 6 3 0.01
ave_val_Loss: 1.23277,test_loss:1.16690
input_size,hidden_size,num_layers,l_rate: 4 6 3 0.005
ave_val_Loss: 1.22815,test_loss:1.16537
input_size,hidden_size,num_layers,l_rate: 4 6 3 0.001
ave_val_Loss: 1.23737,test_loss:1.16845
input_size,hidden_size,num_layers,l_rate: 4 6 4 0.1
ave_val_Loss: 1.23418,test_loss:1.16594
input_size,hidden_size,num_layers,l_rate: 4 6 4 0.01
ave_val_Loss: 1.23896,test_loss:1.17453
input_size,hidden_size,num_layers,l_rate: 4 6 4 0.005
ave_val_Loss: 1.22841,test_loss:1.16491
input_size,hidden_size,num_layers,l_rate: 4 6 4 0.001
ave_val_Loss: 1.27420,test_loss:1.18979


In [13]:
for input_size,hidden_size,num_layers,l_rate in param[96:]:
    if input_size<=hidden_size:
        print("input_size,hidden_size,num_layers,l_rate:",input_size,hidden_size,num_layers,l_rate)
        varl,testl=train_model(input_size,hidden_size,num_layers,l_rate)
        total_para.append((varl,testl,input_size,hidden_size,num_layers,l_rate))
    else:
        continue

input_size,hidden_size,num_layers,l_rate: 6 6 2 0.1
ave_val_Loss: 1.23054,test_loss:1.16841
input_size,hidden_size,num_layers,l_rate: 6 6 2 0.01
ave_val_Loss: 1.23068,test_loss:1.17045
input_size,hidden_size,num_layers,l_rate: 6 6 2 0.005
ave_val_Loss: 1.22361,test_loss:1.16771
input_size,hidden_size,num_layers,l_rate: 6 6 2 0.001
ave_val_Loss: 1.23345,test_loss:1.17861
input_size,hidden_size,num_layers,l_rate: 6 6 3 0.1
ave_val_Loss: 1.23159,test_loss:1.16989
input_size,hidden_size,num_layers,l_rate: 6 6 3 0.01
ave_val_Loss: 1.22684,test_loss:1.16843
input_size,hidden_size,num_layers,l_rate: 6 6 3 0.005
ave_val_Loss: 1.22557,test_loss:1.16835
input_size,hidden_size,num_layers,l_rate: 6 6 3 0.001
ave_val_Loss: 1.34574,test_loss:1.28509
input_size,hidden_size,num_layers,l_rate: 6 6 4 0.1
ave_val_Loss: 1.25026,test_loss:1.16816
input_size,hidden_size,num_layers,l_rate: 6 6 4 0.01
ave_val_Loss: 1.22420,test_loss:1.16852
input_size,hidden_size,num_layers,l_rate: 6 6 4 0.005
ave_val_Loss: 1

In [14]:
for input_size,hidden_size,num_layers,l_rate in param[128:160]:
    if input_size<=hidden_size:
        print("input_size,hidden_size,num_layers,l_rate:",input_size,hidden_size,num_layers,l_rate)
        varl,testl=train_model(input_size,hidden_size,num_layers,l_rate)
        total_para.append((varl,testl,input_size,hidden_size,num_layers,l_rate))
    else:
        continue

In [15]:
for input_size,hidden_size,num_layers,l_rate in param[160:]:
    if input_size<=hidden_size:
        print("input_size,hidden_size,num_layers,l_rate:",input_size,hidden_size,num_layers,l_rate)
        varl,testl=train_model(input_size,hidden_size,num_layers,l_rate)
        total_para.append((varl,testl,input_size,hidden_size,num_layers,l_rate))
    else:
        continue

In [29]:
# print(total_para)
ans = torch.as_tensor(total_para)
print(ans)
print('best parameters:',ans[torch.argmin(ans,0)[0]])

tensor([[1.2383e+00, 1.1643e+00, 3.0000e+00, 3.0000e+00, 2.0000e+00, 1.0000e-01],
        [1.3630e+00, 1.1918e+00, 3.0000e+00, 3.0000e+00, 2.0000e+00, 1.0000e-02],
        [1.2333e+00, 1.1645e+00, 3.0000e+00, 3.0000e+00, 2.0000e+00, 5.0000e-03],
        [1.3980e+00, 1.3443e+00, 3.0000e+00, 3.0000e+00, 2.0000e+00, 1.0000e-03],
        [1.2644e+00, 1.1636e+00, 3.0000e+00, 3.0000e+00, 3.0000e+00, 1.0000e-01],
        [1.2711e+00, 1.1669e+00, 3.0000e+00, 3.0000e+00, 3.0000e+00, 1.0000e-02],
        [1.4762e+00, 1.3259e+00, 3.0000e+00, 3.0000e+00, 3.0000e+00, 5.0000e-03],
        [1.5976e+00, 1.5415e+00, 3.0000e+00, 3.0000e+00, 3.0000e+00, 1.0000e-03],
        [1.2859e+00, 1.1776e+00, 3.0000e+00, 3.0000e+00, 4.0000e+00, 1.0000e-01],
        [1.2458e+00, 1.1654e+00, 3.0000e+00, 3.0000e+00, 4.0000e+00, 1.0000e-02],
        [1.2555e+00, 1.1765e+00, 3.0000e+00, 3.0000e+00, 4.0000e+00, 5.0000e-03],
        [1.3592e+00, 1.3020e+00, 3.0000e+00, 3.0000e+00, 4.0000e+00, 1.0000e-03],
        [1.2421e